In [1]:
from dgp import *

# 1. Simulate or load data
adj = get_graph(800, 1, 2, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=500,
    burnin_R=20, seed=0)

Y_chain = Y_chain[::5]
A_chain = A_chain[::5]
L_chain = L_chain[::5]

100%|██████████| 520/520 [00:43<00:00, 12.00it/s]


In [2]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=200,
    burnin_R=50,
    seed=0
)

{'average': 0.31335625,
 'direct_effect': -0.14082500000000003,
 'spillover_effect': -0.05294999999999994,
 'psi_1_gamma': 0.27003750000000004,
 'psi_0_gamma': 0.4108625000000001,
 'psi_zero': 0.4638125}

In [2]:
from utils import run_pll
from run_pll import run_dr

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain[:4]))]
n_cpu = 4
res_list = run_pll(run_dr, args, processes=n_cpu)
res_list_array = np.array(res_list)


Multiprocessing <function run_dr at 0x113be3b00> in 4 tasks, with 4 processes...
psi_zero: [0.45918527516549057]
beta_hat: 0.44340675182171835
psi_0_gamma: 0.3453576425609083
psi_1_gamma: 0.28825276671437267
average: 0.32156774425344076
direct_effect: -0.05710487584653562
spillover_effect: -0.11382763260458229
psi_zero: [0.5290945749573126]
beta_hat: 0.5000284312872242
psi_0_gamma: 0.4389755408295023
psi_1_gamma: 0.2907492632449398
average: 0.32902880209270713
direct_effect: -0.14822627758456247
spillover_effect: -0.0901190341278103
psi_zero: [0.49376299749670016]
beta_hat: 0.4826423855355928
psi_0_gamma: 0.4824938308029724
psi_1_gamma: 0.28861991368706197
average: 0.34889882062360694
direct_effect: -0.19387391711591045
spillover_effect: -0.011269166693727739
psi_zero: [0.4497551721892136]
beta_hat: 0.45633995627287427
psi_0_gamma: 0.4627720244846745
psi_1_gamma: 0.2584716882472991
average: 0.32097566857525683
direct_effect: -0.20430033623737542
spillover_effect: 0.013016852295460901
M

In [5]:
from run_pll import column_names

ret_mean = res_list_array.mean(axis=0)
ret_std = res_list_array.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean[i]:.5f} ± {ret_std[i]:.5f}")

average: 0.33012 ± 0.01130
direct_effect: -0.15088 ± 0.05810
spillover_effect: -0.05055 ± 0.05281
psi_0_gamma: 0.43240 ± 0.05256
psi_zero: 0.48295 ± 0.03128
psi_1_gamma: 0.28152 ± 0.01334


In [3]:
from drnet import doubly_robust

ret = {
    'average': [],
    'direct_effect': [],
    'spillover_effect': [],
    'psi_0_gamma': [],
    'psi_zero': [],
    "psi_1_gamma":[],
}

for Y, A, L in tqdm(zip(Y_chain, A_chain, L_chain)):
    ret_i = doubly_robust(A, L, Y, adj, treatment_allocation=0.7, num_rep=1000)
    ret['average'].append(ret_i['average'])
    ret['direct_effect'].append(ret_i['direct_effect'])
    ret['spillover_effect'].append(ret_i['spillover_effect'])
    ret['psi_0_gamma'].append(ret_i['psi_0_gamma'])
    ret['psi_zero'].append(ret_i['psi_zero'])
    ret['psi_1_gamma'].append(ret_i['psi_1_gamma'])
    
print('Average effect:', np.mean(ret['average']))
print('Direct effect:', np.mean(ret['direct_effect']))
print('Spillover effect:', np.mean(ret['spillover_effect']))
print('Psi_0_gamma:', np.mean(ret['psi_0_gamma']))
print('Psi_zero:', np.mean(ret['psi_zero']))
print('Psi_1_gamma:', np.mean(ret['psi_1_gamma']))

1it [00:18, 18.30s/it]

psi_zero: [0.5290945749573126]
beta_hat: 0.5000284312872242
psi_0_gamma: 0.4389755408295023
psi_1_gamma: 0.2907492632449398
average: 0.32902880209270713
direct_effect: -0.14822627758456247
spillover_effect: -0.0901190341278103


2it [00:37, 18.57s/it]

psi_zero: [0.45918527516549057]
beta_hat: 0.44340675182171835
psi_0_gamma: 0.3453576425609083
psi_1_gamma: 0.28825276671437267
average: 0.32156774425344076
direct_effect: -0.05710487584653562
spillover_effect: -0.11382763260458229


3it [00:55, 18.51s/it]

...

psi_zero: [0.4412020012100239]
beta_hat: 0.43292508653096556
psi_0_gamma: 0.3970689892459853
psi_1_gamma: 0.3126712764336825
average: 0.33823610968030177
direct_effect: -0.08439771281230285
spillover_effect: -0.04413301196403857


13it [04:06, 19.75s/it]

psi_zero: [0.44336855314100077]
beta_hat: 0.4267934678468314
psi_0_gamma: 0.37686688784902755
psi_1_gamma: 0.2541419781746574
average: 0.29415521094524166
direct_effect: -0.12272490967437016
spillover_effect: -0.06650166529197321


13it [04:06, 19.00s/it]


KeyboardInterrupt: 

## Dense network

In [ ]:
from dgp import *

# 1. Simulate or load data
adj = get_graph(2000, 8, 10, seed=1)

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     # shape (10,)

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=500,
    burnin_R=20, seed=0)

Y_chain = Y_chain[::5]
A_chain = A_chain[::5]
L_chain = L_chain[::5]

In [ ]:
from agcEffect import *

agc_effect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=200,
    burnin_R=50,
    seed=0
)

In [ ]:
from drnet import doubly_robust

ret = {
    'average': [],
    'direct_effect': [],
    'spillover_effect': [],
    'psi_0_gamma': [],
    'psi_zero': [],
    "psi_1_gamma":[],
}

for Y, A, L in tqdm(zip(Y_chain, A_chain, L_chain)):
    ret_i = doubly_robust(A, L, Y, adj, treatment_allocation=0.7, num_rep=1000)
    ret['average'].append(ret_i['average'])
    ret['direct_effect'].append(ret_i['direct_effect'])
    ret['spillover_effect'].append(ret_i['spillover_effect'])
    ret['psi_0_gamma'].append(ret_i['psi_0_gamma'])
    ret['psi_zero'].append(ret_i['psi_zero'])
    ret['psi_1_gamma'].append(ret_i['psi_1_gamma'])
    
print('Average effect:', np.mean(ret['average']))
print('Direct effect:', np.mean(ret['direct_effect']))
print('Spillover effect:', np.mean(ret['spillover_effect']))
print('Psi_0_gamma:', np.mean(ret['psi_0_gamma']))
print('Psi_zero:', np.mean(ret['psi_zero']))
print('Psi_1_gamma:', np.mean(ret['psi_1_gamma']))